# Installing neccessary packages


# Install your packages here

In [3]:
install.packages('jsonlite')
install.packages('rvest')
install.packages('tidyverse')
install.packages('lubridate')
install.packages('httr')
install.packages('fastDummies')
install.packages('tidymodels')
install.packages('distributions3')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("jsonlite"):
“installation of package ‘jsonlite’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("httr"):
“installation of package ‘httr’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Load the libraries here

In [7]:
library(jsonlite)
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)
library(fastDummies)
library(tidymodels)
library(distributions3)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ purrr::flatten()        masks jsonlite::flatten()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.1     ✔ rsample      1.1.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.3     ✔ workflows    1.0.0
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.1     ✔ yardstick    1.1.0
✔ recipes      1.0.1     

── Conflicts ──────

# Custom functions

In [2]:
# function to trim an array
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

# function that adds the ternary operator to R

ternary = function(expression_1, expression_2, condition) {
    if(condition) {
        expression_1
    } else {
        expression_2
    }
}

ternary_v = Vectorize(ternary)

# split

strsplit_custom = function (string, splitter, index) {
    strsplit(string, splitter)[[1]][index]
}

strsplit_custom_v = Vectorize(strsplit_custom)

# convert month to decimal

convert_month_to_decimal = function(month) {
    num = 0
    
    for (i in 1:12) {
        if(month == switch(i,
                          'Jan',
                          'Feb',
                          'Mar',
                          'Apr',
                          'May',
                          'Jun',
                          'Jul',
                          'Aug',
                          'Sep',
                          'Oct',
                          'Nov',
                          'Dec')
          ) {
            num = i
            break;
        }
    }
    return(i)
}

convert_month_to_decimal_v = Vectorize(convert_month_to_decimal)

# convert from named list to number

toNumber = function(value) {
    return(value[1])
}

toNumber_v = Vectorize(toNumber)

# is Neutral

isNeutral = function(country, opponent, game_name) {
    
    HOSTCOUNTRY = NULL
    
    if(game_name == 'World Cup 2010') {
        HOSTCOUNTRY = 'South Africa'
    } else if(game_name == 'World Cup 2014') {
        HOSTCOUNTRY = 'Brazil'
    } else if(game_name == 'World Cup 2018') {
        HOSTCOUNTRY = 'Russia'
    } else if(game_name == 'World Cup 2022') {
        HOSTCOUNTRY = 'Qatar'
    } 
    
    if (!is.null(HOSTCOUNTRY)) {
        venue = 'N'
        if(country != HOSTCOUNTRY & opponent != HOSTCOUNTRY) {
            return('N')
        } else {
            return('H/A')
        }
    } else {
        return ('H/A')
    }
}
    
isNeutral_v = Vectorize(isNeutral)
    
# test_date function
    
test_date = function(date_param) {
    i = 1;
    
    date_list = unique(fifa_ranking1$fifa_date)
    
    # loops through a date_list which we got from the fifa ranking dataset
    for(i in 1:length(date_list)) {
        # test if the supplied date_param is smaller then the fifa_date inside date_list
        # if it is, the loop will break, which means it stops and jumps out of the loop
        if(date_list[i] > date_param) {
            break;
        }
    }
    # This is were the execution continous when we break out of the loop
    # This is also the place where we continue execution in case the for loop finishes through all iterations
    if(i != 1) {
        # if i does not stay 1 it should always return the value that has come before it
        return(date_list[i-1])
    } else {
        # if i does stay 1 it should always return 1
        return(date_list[1])
    }
}

# We vectorize the function to make it usable within any function that works with vectors and not single values
test_date_v = Vectorize(test_date)
    
# getFifaRanking
getFifaRanking = function(country, date) {
    
    fifa_ranking1 %>%
        filter(Land == country) %>%
        filter(fifa_date == date) %>%
        pluck(2) -> temp
    #print(as.numeric(unlist(temp)))
    
    as.numeric(temp)
}

getFifaRanking_v = Vectorize(getFifaRanking)

# getProbabilities
    
getProbabilities = function(df, model, team, opponent, range) {
    
    df %>%
        filter(country == team) %>%
        filter(Opponent == opponent) -> new_df
    
    dataFrame = data.frame(rank_opponent = new_df$rank_opponent, Venue_H = new_df$Venue_H, Venue_N = new_df$Venue_N, Rank = new_df$Rank)
    
    lambda = predict(model, newdata = dataFrame, type = "response")
    
    p = Poisson(lambda)
    
    prob = pdf(p, range)
    
    return(prob)
}
    
# get goals
    
getGoals = function(df, model, team, opponent) {
    df %>%
        filter(country == team) %>%
        filter(Opponent == opponent) -> new_df
    
    dataFrame = data.frame(rank_opponent = new_df$rank_opponent, Venue_H = new_df$Venue_H, Venue_N = new_df$Venue_N, Rank = new_df$Rank)
    
    lambda = predict(model, newdata = dataFrame, type = "response")
    return (rpois(1, lambda))
}
    
# get probability data frame
getProbabilityMatrix = function(df, model, team, opponent, range) {
    res <- outer(getProbabilities(df, model, team, opponent, range), getProbabilities(df, model, opponent, team, range))
    df = data.frame(round(res, digits = 2))
    
    return (df)
}
   
# getDraw probability
    
getDraw = function(data) {
    draw = 0
    for (i in 1:length(data[[1]])) {
        draw = draw + data[i, i]
    }
    return (draw)
}
    
# getWin probability

getWin = function(data) {
    win = 0
    for (i in 1:length(data[[1]])) {
        j = 1
        #print("here")
        for (j in (1+i):length(data[[1]])-1) {
            if(j != i) {
                #print(paste0('data[', j, ',', i, ']'))
                #print(data[j, i])
                win = win + data[j, i]
            }
            
        }
    }
    return (win)
}
    
# getLoose probability

getLoose = function(data) {
    win = 0
    for (i in 1:length(data[[1]])) {
        j = 1
        for (j in 1:length(data[[1]])) {
            if(j != i & j < i) {
                #print(paste0('data[', j, ',', i, ']'))
                #print(data[j, i])
                win = win + data[j, i]
            }
            
        }
    }
    return (win)
}


# Scrape Teilnehmer von WM

In [3]:
scrapeColumnsWM = function(game_name) {

  # base url

  #url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'
  url = 'https://www.transfermarkt.com/weltmeisterschaft-2018/teilnehmer/pokalwettbewerb/WM18/saison_id/2017'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data
      
      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

# create Dataframe from scrapeColumnsWM

In [4]:
create_dataframeWM = function(name) {
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [12]:
df_wm = create_dataframeWM('CLUBS STARTING INTO TOURNAMENT AT A LATER POINT')
df_wm

[1] "Column 1 is not a valid value"


Club,Squad,&oslash-Age,Market Value,&oslash-Market Value,slug,club_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
England,28,,€1.34bn,€47.71m,england,3299
Brazil,26,,€1.05bn,€40.54m,brasilien,3439
Portugal,25,,€938.00m,€37.52m,portugal,3300
France,23,,€909.50m,€39.54m,frankreich,3377
Germany,24,,€818.00m,€34.08m,deutschland,3262
Spain,25,,€790.00m,€31.60m,spanien,3375
Argentina,28,,€671.00m,€23.96m,argentinien,3437
Belgium,30,,€564.50m,€18.82m,belgien,3382
Uruguay,26,,€375.40m,€14.44m,uruguay,3449


In [13]:
teilnehmer = df_wm$Club
teilnehmer

[1] "England"      "Brazil"       "Portugal"     "France"       "Germany"     
 [6] "Spain"        "Argentina"    "Belgium"      "Uruguay"      "Croatia"     
[11] "Serbia"       "Denmark"      "Senegal"      "Colombia"     "Poland"      
[16] "Nigeria"      "Morocco"      "Switzerland"  "Sweden"       "Mexico"      
[21] "South Korea"  "Egypt"        "Russia"       "Japan"        "Iran"        
[26] "Tunisia"      "Peru"         "Australia"    "Iceland"      "Saudi Arabia"
[31] "Panama"       "Costa Rica"

# Scrape Columns
We wrote a function, that takes a url to scrape all tables

In [14]:
scrapeGame = function(year, country, club_id, game_name) {

  # base url

  url = paste0("https://www.transfermarkt.com/",
               country,
               "/spielplan/verein/",
               club_id,
               "/plus/0?saison_id=", year)
    
  # print url
  print(url)
    
  # get status code
  res = httr::GET(url)

  # read response
  code = httr::status_code(res)
    
  # print status code
  print(code)
    
  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    if(length(html_elements(item, '.table-header') %>% html_children())!= 0) {
      item %>%
        html_elements('.table-header a img') %>%
        html_attr('title') -> name

      #print(name)
    } else {
      item %>%
        html_elements('.table-header') %>%
        html_text() %>%
        trimStringArray() -> name
        
        #print(name)
    }

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames
      
      # add game_name to columnNames
      
      columnNames = append(columnNames, 'game_name')
      columnNames = append(columnNames, 'country')

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              #print(paste0("Column ", i, " is not a valid value"))
          }

      }
        
      # add game_name to columnValues
      
      columnValues = append(columnValues, name)
      
      # add country name to columnValues
      columnValues = append(columnValues, country)
      
      # return data frame
      # change column names by replacing whitespace with underscore
      columnNames = gsub(" ", "_", columnNames)

      # create a dataframe with first colum and append the rest (workaround)
      df = data.frame(columnValues[[1]])

      for (i in 2:length(columnValues)) {
          df[i] = columnValues[[i]]
      }

      names(df) = columnNames
      return(df)
      
  } else {
      return(data.frame())
  }
}

# Testing Section

In [15]:
#scrapeGame(year = '2009', country = 'England', club_id = 3299, game_name = "weltmeisterschaft [0-9]+")

scrapeGameByYear = function(year, country) {
    
    df_wm %>%
        filter(Club == country) -> tempDf
    
    tempDf$club_id -> club_id
    
    #pmap_df(list('2015', 'England', '3299', c('weltmeisterschaft [0-9]+', 'freundschaftsspiele', 'em-qualifikation', 'wm-qualifikation europa', 'europameisterschaft [0-9]+')), scrapeGame)
    return(pmap_df(list(year, country, club_id,
                        c(
                            'world cup [0-9]+',
                            'international friendlies'
                            #'european qualifiers',
                            #'world cup qualification africa'
                            #'euro [0-9]+'
                            #'copa'
                            #'confederations cup [0-9]+'
                            #'ostasienmeisterschaft',
                            #'wm-qualifikation asien',
                            #'asienmeisterschaft [0-9]+',
                            #'asienmeisterschaft qualifikation'
                            #'africa cup of nations'
                            #'fifa arab cup',
                            #'gold cup [0-9]+'
                        )), scrapeGame))
}

In [16]:
scrapeSomething = function(country) {
    return(map2_df(seq('2009', '2021', by=1), country, scrapeGameByYear))
}

## Scrape world cup & friendlies for all 32 countries

In [17]:
complete_df = map_df(teilnehmer, scrapeSomething)

In [74]:
complete_df

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group C,"Sat Jun 12, 2010",8:30 PM,H,,United States,4-4-2 double 6,38.646,1:1,World Cup 2010,England
Group C,"Fri Jun 18, 2010",8:30 PM,H,,Algeria,4-4-2 double 6,64.100,0:0,World Cup 2010,England
Group C,"Wed Jun 23, 2010",4:00 PM,A,,Slovenia,4-4-2 double 6,36.893,0:1,World Cup 2010,England
Round of 16,"Sun Jun 27, 2010",4:00 PM,A,,Germany,4-4-2 double 6,40.510,4:1,World Cup 2010,England
NA,"Wed Mar 3, 2010",9:00 PM,H,,Egypt,4-4-2 double 6,80.602,3:1,International Friendlies,England
NA,"Mon May 24, 2010",9:00 PM,H,,Mexico,4-2-3-1,88.638,3:1,International Friendlies,England
NA,"Sun May 30, 2010",2:15 PM,H,,Japan,4-4-2 double 6,15.326,2:1,International Friendlies,England
NA,"Wed Aug 11, 2010",9:00 PM,H,,Hungary,4-2-3-1,72.024,2:1,International Friendlies,England
NA,"Wed Nov 17, 2010",9:00 PM,H,,France,4-2-3-1,85.495,1:2,International Friendlies,England


# Split Result in Goals

In [16]:
#df_football %>%
#    mutate(goals_scored_england = ternary_v(strsplit_custom_v(Ergebnis, ':', 1), strsplit_custom_v(Ergebnis, ':', 2), (Heimmannschaft=='England'))) -> df_football_goals
#df_football_goals

## We save all data for the WorldCups&friendlies as csv

In [75]:
#complete_df
write.csv2(complete_df, file='./wm2018_football_en.csv', row.names = F)

# Testing Environment

## Get Dataset from our github repository

In [8]:
data_set2018 = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/data/wm2018_football_en.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")

## Preprocessing

In [9]:
#data_set2018

fifa_ranking = read.csv(
    "https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv",
    header = TRUE,
    sep = ",",
    stringsAsFactors=FALSE,
    fileEncoding="latin1"
)


fifa_ranking %>%
separate(Rang, c('Rang', 'Sym')) %>%
select(-'Sym') %>%
mutate(fifa_date = ymd(Datum)) %>%
select(-c('Datum')) -> fifa_ranking1

data_set2018 %>%
    # reduce dataset to relevant columns
    select(c('Date', 'Venue', 'Opponent', 'Result', 'game_name', 'country')) %>%
    # separate Result column into result_left and result_right
    separate(Result, c('result_left', 'result_right')) %>%
    # create a new column called goals_country
    mutate(goals_country = ternary_v(result_left, result_right, Venue=='H')) %>%
    # create a new column called goals_enemy
    mutate(goals_enemy = ternary_v(result_left, result_right, Venue=='A')) %>%
    # separate the Date column into weekday, month, day and year
    separate(Date, c('weekday', 'month', 'day', 'year'), sep=' ', remove=F) %>%
    # after separation the column day contains a comma after its value ('11,'), which we get rid of as follows
    separate(day, c('day')) %>%
    # then we convert the month string literal to its represantative numerical value as follows
    mutate(month = convert_month_to_decimal_v(month)) %>%
    # using year, month (converted to numerical) and day we create a date object of this format (yyyy-mm-dd)
    mutate(date_of_play = ymd(paste0(year, '-', month, '-', day))) %>%
    # then we deselect the column names as stated below to finish up the new cleaned data_set
    select(-c('Date', 'weekday', 'month', 'day', 'year')) %>%
    # create a column called fifa_date using the test_date_v function
    mutate(fifa_date = as_date(test_date_v(date_of_play))) -> df

# merge the fifa ranking dataset with df using country/Land and fifa_date/fifa_date as merge points
merge(df, fifa_ranking1, by.x=c('country', 'fifa_date'), by.y=c('Land', 'fifa_date')) %>%
# create a new column called rank_opponent using the getFifaRanking_v function
mutate(rank_opponent = getFifaRanking_v(Opponent, fifa_date)) %>%
# convert the rank_opponent column from named list to number 
mutate(rank_opponent = toNumber_v(rank_opponent)) %>%
# create new column isNeutral using the isNeutral_v function
mutate(isNeutral = isNeutral_v(country, Opponent, game_name)) %>%
# convert Venue to N if it is a neutral game
mutate(Venue = ifelse(isNeutral == 'N', 'N', Venue)) %>%
# turn goals_country to numerical
mutate(goals_country = as.numeric(goals_country)) %>%
# turn goals_enemy to numerical
mutate(goals_enemy = as.numeric(goals_enemy)) %>%
# turn Rank to numerical
mutate(Rank = as.numeric(Rang)) %>%
# turn Venue into dummy variables
dummy_cols(select_columns = 'Venue') %>%
# deselect irrelevant columns
select(-c('isNeutral', 'Punkte', 'Rang')) -> df


Warning message:
“Expected 2 pieces. Additional pieces discarded in 31 rows [51, 53, 124, 371, 374, 420, 474, 531, 533, 534, 605, 653, 686, 759, 760, 766, 767, 768, 891, 899, ...].”
Warning message:
“Expected 1 pieces. Additional pieces discarded in 2393 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


In [10]:
df

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<int>,<int>,<int>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,44,8,0,1,0
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,78,8,0,1,0
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,5,9,1,0,0
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,87,7,0,1,0
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,63,7,0,1,0
Argentina,2010-05-26,N,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,21,7,0,0,1
Argentina,2010-05-26,N,South Korea,4,1,World Cup 2010,4,1,2010-06-17,47,7,0,0,1
Argentina,2010-05-26,N,Greece,0,2,World Cup 2010,2,0,2010-06-22,13,7,0,0,1
Argentina,2010-05-26,N,Mexico,3,1,World Cup 2010,3,1,2010-06-27,17,7,0,0,1


## Split data into train and test data

In [11]:
df %>%
    filter(game_name != 'World Cup 2018') -> train

df %>%
    filter(game_name == 'World Cup 2018') -> test

test %>%
    filter(country == 'Germany')

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<int>,<int>,<int>
Germany,2018-06-07,N,Mexico,0,1,World Cup 2018,0,1,2018-06-17,15,1,0,0,1
Germany,2018-06-07,N,Sweden,2,1,World Cup 2018,2,1,2018-06-23,24,1,0,0,1
Germany,2018-06-07,N,South Korea,2,0,World Cup 2018,0,2,2018-06-27,57,1,0,0,1


## Model training

In [12]:
train

model = 
    glm(
        goals_country ~ Rank + rank_opponent + Venue_H + Venue_N,
        train,
        family = poisson(link = 'log')
       )
summary(model)

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<int>,<int>,<int>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,44,8,0,1,0
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,78,8,0,1,0
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,5,9,1,0,0
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,87,7,0,1,0
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,63,7,0,1,0
Argentina,2010-05-26,N,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,21,7,0,0,1
Argentina,2010-05-26,N,South Korea,4,1,World Cup 2010,4,1,2010-06-17,47,7,0,0,1
Argentina,2010-05-26,N,Greece,0,2,World Cup 2010,2,0,2010-06-22,13,7,0,0,1
Argentina,2010-05-26,N,Mexico,3,1,World Cup 2010,3,1,2010-06-27,17,7,0,0,1



Call:
glm(formula = goals_country ~ Rank + rank_opponent + Venue_H + 
    Venue_N, family = poisson(link = "log"), data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7570  -1.1975  -0.1849   0.5668   3.7348  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.2089256  0.0406895   5.135 2.83e-07 ***
Rank          -0.0095337  0.0008310 -11.473  < 2e-16 ***
rank_opponent  0.0065607  0.0004345  15.100  < 2e-16 ***
Venue_H        0.2846201  0.0377181   7.546 4.49e-14 ***
Venue_N        0.1746353  0.0778739   2.243   0.0249 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3017.1  on 2115  degrees of freedom
Residual deviance: 2637.3  on 2111  degrees of freedom
  (149 observations deleted due to missingness)
AIC: 6622.8

Number of Fisher Scoring iterations: 5


## Model testing

In [69]:
getGroupPlaySummary = function(df, model, team_A, team_B, team_C, team_D) {
    
    #print(df)
    
    team_A_points = 3
    team_B_points = 3
    team_C_points = 3
    team_D_points = 3
    
    result = c()
    country = c()
    opponent = c()
    
    getResult = function(df, model, team, opponent) {
        
        team_goals = getGoals(df, model, team, opponent)
        
        opponent_goals = getGoals(df, model, opponent, team)
        
        #print(paste0(team, '-', opponent, ':', ' ', team_goals, ':', opponent_goals))
        
        result_value = paste0(team_goals, ':', opponent_goals)
        
        result = append(result, result_value)
        
        #result = append(result, c(paste0(team_goals, ':', opponent_goals)))
        
        return (c('team'=team_goals, 'opponent'=opponent_goals))
    }
    
    update_points = function(goals, team, opponent, team_points_param, opponent_points_param) {
        list_ = goals

        #print('list_')
        #print(list_)

        team_points = 
            team_points_param + 
            unname(list_['team']) -
            unname(list_['opponent'])

        #print('team_points')
        #print(team_points)

        opponent_points = 
                opponent_points_param + 
                unname(list_['opponent']) -
                unname(list_['team'])

        #print('opponent_points')
        #print(opponent_points)

        return (c(team_points=team_points, opponent_points=opponent_points))
    }
    
    # team A - team B
    goals = getResult(df, model, team_A, team_B)
    points = update_points(goals, team_A, team_B, team_A_points, team_B_points)
    #print('points')
    #print(points)
    team_A_points = unname(points['team_points'])
    team_B_points = unname(points['opponent_points'])
    
    result = append(result, paste0(unname(goals['team']), ':', paste0(unname(goals['opponent']))))
    country = append(country, team_A)
    opponent = append(opponent, team_B)
        
    # team A - team C
    goals = getResult(df, model, team_A, team_C)
    points = update_points(goals, team_A, team_C, team_A_points, team_C_points)
    team_A_points = unname(points['team_points'])
    team_C_points = unname(points['opponent_points'])
    
    result = append(result, paste0(unname(goals['team']), ':', paste0(unname(goals['opponent']))))
    country = append(country, team_A)
    opponent = append(opponent, team_C)
    
    # team A - team D
    goals = getResult(df, model, team_A, team_D)
    points = update_points(goals, team_A, team_D, team_A_points, team_D_points)
    team_A_points = unname(points['team_points'])
    team_D_points = unname(points['opponent_points'])
    
    result = append(result, paste0(unname(goals['team']), ':', paste0(unname(goals['opponent']))))
    country = append(country, team_A)
    opponent = append(opponent, team_D)
    
    # team B - team C
    goals = getResult(df, model, team_B, team_C)
    points = update_points(goals, team_B, team_C, team_B_points, team_C_points)
    team_B_points = unname(points['team_points'])
    team_C_points = unname(points['opponent_points'])
    
    result = append(result, paste0(unname(goals['team']), ':', paste0(unname(goals['opponent']))))
    country = append(country, team_B)
    opponent = append(opponent, team_C)
    
    # team B - team D
    goals = getResult(df, model, team_B, team_D)
    points = update_points(goals, team_B, team_D, team_B_points, team_D_points)
    team_B_points = unname(points['team_points'])
    team_D_points = unname(points['opponent_points'])
    
    result = append(result, paste0(unname(goals['team']), ':', paste0(unname(goals['opponent']))))
    country = append(country, team_B)
    opponent = append(opponent, team_D)
    
    # team C - team D
    goals = getResult(df, model, team_B, team_D)
    points = update_points(goals, team_C, team_D, team_C_points, team_D_points)
    team_C_points = unname(points['team_points'])
    team_D_points = unname(points['opponent_points'])
    
    result = append(result, paste0(unname(goals['team']), ':', paste0(unname(goals['opponent']))))
    country = append(country, team_C)
    opponent = append(opponent, team_D)
    
    #print(result)
    
    data.frame(Country=c(team_A, team_B, team_C, team_D), Points=c(team_A_points, team_B_points, team_C_points, team_D_points)) -> ranking
    ranking %>%
        arrange(desc(Points)) -> ranking
    
    return (list(
        ranking = ranking,
        resultTable = data.frame(Country=country, Opponent=opponent, Result=result)))
}

In [82]:
first_place = 0
second_place = 0
third_place = 0
fourth_place = 0

for(i in 1:1000) {
    summary = getGroupPlaySummary(test, model, 'Russia', 'Uruguay', 'Saudi Arabia', 'Egypt')
    
    if(summary$ranking[1,1] == 'Russia') {
        first_place = first_place + 1
    } else if (summary$ranking[2,1] == 'Russia') {
        second_place = second_place + 1
    } else if (summary$ranking[3,1] == 'Russia') {
        third_place = third_place + 1
    } else if (summary$ranking[4,1] == 'Russia') {
        fourth_place = fourth_place + 1
    }
}

print('first_place')
print(first_place)

print('second_place')
print(second_place)

print('third_place')
print(third_place)

print('fourth_place')
print(fourth_place)


[1] "first_place"
[1] 109
[1] "second_place"
[1] 271
[1] "third_place"
[1] 306
[1] "fourth_place"
[1] 314


In [59]:
# Russia and SaudiArabia
russia_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 1, Venue_N= 0, Rank = 70)
saudiArabia_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 67)

# Egypt and Uruguay
egypt_uruguay = data.frame(rank_opponent = 14, Venue_H = 0, Venue_N= 1, Rank = 45)
uruguay_egypt = data.frame(rank_opponent = 45, Venue_H = 0, Venue_N= 1, Rank = 14)

# Russia and Egypt
russia_egypt = data.frame(rank_opponent = 14, Venue_H = 1, Venue_N= 0, Rank = 70)
egypt_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 14)

# Uruguay and Saudi Arabia
uruguay_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 0, Venue_N= 1, Rank = 14)
saudiArabia_uruguay = data.frame(rank_opponent = 14, Venue_H = 0, Venue_N= 1, Rank = 67)

# Uruguay and Russia
russia_uruguay = data.frame(rank_opponent = 14, Venue_H = 1, Venue_N= 0, Rank = 70)
uruguay_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 14)

# Saudi Arabia and Egypt
egypt_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 0, Venue_N= 1, Rank = 45)
saudiArabia_russia = data.frame(rank_opponent = 45, Venue_H = 0, Venue_N= 1, Rank = 67)

test %>%
    filter(country == 'Uruguay') %>%
    filter(Opponent == 'Saudi Arabia')

print("Get win probability of Saudi Arabia against Uruguay")
getWin(getProbabilityMatrix(test, model, 'Saudi Arabia', 'Uruguay', 0:6))

print("Get win probability of Uruguay against Saudi Arabia")
getWin(getProbabilityMatrix(test, model, 'Uruguay', 'Saudi Arabia', 0:6))

test_function = function(df, model, team, opponent, range) {
    print(getProbabilities(test, model, team, opponent, 0:6))
    print(getProbabilities(test, model, opponent, team, 0:6))
    print(getGoals(test, model, team, opponent))
    print(getProbabilityMatrix(test, model, team, opponent, 0:6))
}

test_function(test, model, 'Uruguay', 'Saudi Arabia', 0:6)


country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<int>,<int>,<int>
Uruguay,2018-06-07,N,Saudi Arabia,1,0,World Cup 2018,1,0,2018-06-20,67,14,0,0,1


[1] "Get win probability of Saudi Arabia against Uruguay"


[1] 0.14

[1] "Get win probability of Uruguay against Saudi Arabia"


[1] 0.64

[1] 0.13628228 0.27161426 0.27066727 0.17981572 0.08959440 0.03571281 0.01186277
[1] 0.4277152914 0.3632575327 0.1542568593 0.0436699889 0.0092722032
[6] 0.0015749718 0.0002229366
[1] 2
    X1   X2   X3   X4 X5 X6 X7
1 0.06 0.05 0.02 0.01  0  0  0
2 0.12 0.10 0.04 0.01  0  0  0
3 0.12 0.10 0.04 0.01  0  0  0
4 0.08 0.07 0.03 0.01  0  0  0
5 0.04 0.03 0.01 0.00  0  0  0
6 0.02 0.01 0.01 0.00  0  0  0
7 0.01 0.00 0.00 0.00  0  0  0


## Modelling - Data Cleaning and Transformation

In [14]:
data_set = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/world_cup%26friendlies_football_en.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")

In [17]:
data_set

Matchday,Date,Time,Venue,Ranking,Opponent,System_of_play,Attendance,Result,game_name,country
<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Group C,"Sat Jun 12, 2010",8:30 PM,H,NA,United States,4-4-2 double 6,38.646,1:1,World Cup 2010,England
Group C,"Fri Jun 18, 2010",8:30 PM,H,NA,Algeria,4-4-2 double 6,64.100,0:0,World Cup 2010,England
Group C,"Wed Jun 23, 2010",4:00 PM,A,NA,Slovenia,4-4-2 double 6,36.893,0:1,World Cup 2010,England
Round of 16,"Sun Jun 27, 2010",4:00 PM,A,NA,Germany,4-4-2 double 6,40.510,4:1,World Cup 2010,England
NA,"Wed Mar 3, 2010",9:00 PM,H,NA,Egypt,4-4-2 double 6,80.602,3:1,International Friendlies,England
NA,"Mon May 24, 2010",9:00 PM,H,NA,Mexico,4-2-3-1,88.638,3:1,International Friendlies,England
NA,"Sun May 30, 2010",2:15 PM,H,NA,Japan,4-4-2 double 6,15.326,2:1,International Friendlies,England
NA,"Wed Aug 11, 2010",9:00 PM,H,NA,Hungary,4-2-3-1,72.024,2:1,International Friendlies,England
NA,"Wed Nov 17, 2010",9:00 PM,H,NA,France,4-2-3-1,85.495,1:2,International Friendlies,England


In [18]:
data_set_reduced = data_set %>%
    select(c('Date', 'Venue', 'Opponent', 'Result', 'game_name', 'country'))

## Obtain goals of enemy and country

In [19]:
data_set_reduced %>%
    separate(Result, c('result_left', 'result_right')) %>%
    mutate(goals_country = ternary_v(result_left, result_right, Venue=='H')) %>%
    mutate(goals_enemy = ternary_v(result_left, result_right, Venue=='A')) -> data_set_increased

data_set_increased

Warning message:
“Expected 2 pieces. Additional pieces discarded in 38 rows [51, 53, 212, 304, 307, 420, 474, 531, 533, 534, 589, 617, 618, 660, 693, 832, 833, 839, 840, 841, ...].”


Date,Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Sat Jun 12, 2010",H,United States,1,1,World Cup 2010,England,1,1
"Fri Jun 18, 2010",H,Algeria,0,0,World Cup 2010,England,0,0
"Wed Jun 23, 2010",A,Slovenia,0,1,World Cup 2010,England,1,0
"Sun Jun 27, 2010",A,Germany,4,1,World Cup 2010,England,1,4
"Wed Mar 3, 2010",H,Egypt,3,1,International Friendlies,England,3,1
"Mon May 24, 2010",H,Mexico,3,1,International Friendlies,England,3,1
"Sun May 30, 2010",H,Japan,2,1,International Friendlies,England,2,1
"Wed Aug 11, 2010",H,Hungary,2,1,International Friendlies,England,2,1
"Wed Nov 17, 2010",H,France,1,2,International Friendlies,England,1,2


## This funtion converts string literals of abbrevated month names and turns them into their representative numerical value

In [21]:
data_set_increased %>%
    # separate the Date column into weekday, month, day and year
    separate(Date, c('weekday', 'month', 'day', 'year'), sep=' ', remove=F) %>%
    # after separation the column day contains a comma after its value ('11,'), which we get rid of as follows
    separate(day, c('day')) %>%
    # then we convert the month string literal to its represantative numerical value as follows
    mutate(month = convert_month_to_decimal_v(month)) %>%
    # using year, month (converted to numerical) and day we create a date object of this format (yyyy-mm-dd)
    mutate(date_of_play = ymd(paste0(year, '-', month, '-', day))) %>%
    # then we deselect the column names as stated below to finish up the new cleaned data_set
    select(-c('Date', 'weekday', 'month', 'day', 'year')) -> data_set_cleaned1

# -> The warning message should be no concern, as it simply states that the function separate(day, c('day')) requires an additional column name to store the leftover of the separation
# Since we have no use of this leftover we leave it to the function to simply discard it for us.
data_set_cleaned1

Warning message:
“Expected 1 pieces. Additional pieces discarded in 2399 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17


## This function gives us the current fifa update date of any date we supply to it. It is necessary to find out the current fifa ranking for each match 

## Apply the function test_date_v on the dataset

In [94]:
data_set_cleaned1 %>%
    mutate(fifa_date = as_date(test_date_v(date_of_play))) -> data_set_fifa_date

In [24]:
data_set_fifa_date

Venue,Opponent,result_left,result_right,game_name,country,goals_country,goals_enemy,date_of_play,fifa_date
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>
H,United States,1,1,World Cup 2010,England,1,1,2010-06-12,2010-05-26
H,Algeria,0,0,World Cup 2010,England,0,0,2010-06-18,2010-05-26
A,Slovenia,0,1,World Cup 2010,England,1,0,2010-06-23,2010-05-26
A,Germany,4,1,World Cup 2010,England,1,4,2010-06-27,2010-05-26
H,Egypt,3,1,International Friendlies,England,3,1,2010-03-03,2010-03-03
H,Mexico,3,1,International Friendlies,England,3,1,2010-05-24,2010-04-28
H,Japan,2,1,International Friendlies,England,2,1,2010-05-30,2010-05-26
H,Hungary,2,1,International Friendlies,England,2,1,2010-08-11,2010-08-11
H,France,1,2,International Friendlies,England,1,2,2010-11-17,2010-11-17


## join both data frames using fifa_date

In [25]:
merge(data_set_fifa_date, fifa_ranking1, by.x=c('country', 'fifa_date'), by.y=c('Land', 'fifa_date')) -> newest_df

In [27]:
newest_df %>% 
    mutate(rank_opponent = getFifaRanking_v(Opponent, fifa_date)) -> somethingDf


In [28]:
somethingDf

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang,Punkte,rank_opponent
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<named list>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8,1085,44
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8,1082,78
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9,1087,5
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7,1084,87
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7,1084,63
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7,1076,21
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7,1076,47
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7,1076,13
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7,1076,17


## This function is necessary to convert the column rank_opponent from named_list to number

## Functions for Venue transformation

In [31]:
somethingDf %>%
    mutate(rank_opponent = toNumber_v(rank_opponent)) -> test

unique(test$game_name)
test

[1] "International Friendlies" "World Cup 2010"          
[3] "World Cup 2014"           "World Cup 2018"          
[5] "World Cup 2022"

country,fifa_date,Venue,Opponent,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,Rang,Punkte,rank_opponent
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>
Argentina,2009-12-16,H,Costa Rica,3,2,International Friendlies,3,2,2010-01-27,8,1085,44
Argentina,2010-02-03,H,Jamaica,2,1,International Friendlies,2,1,2010-02-11,8,1082,78
Argentina,2010-03-03,A,Germany,0,1,International Friendlies,1,0,2010-03-03,9,1087,5
Argentina,2010-04-28,H,Haiti,4,0,International Friendlies,4,0,2010-05-05,7,1084,87
Argentina,2010-04-28,H,Canada,5,0,International Friendlies,5,0,2010-05-24,7,1084,63
Argentina,2010-05-26,H,Nigeria,1,0,World Cup 2010,1,0,2010-06-12,7,1076,21
Argentina,2010-05-26,H,South Korea,4,1,World Cup 2010,4,1,2010-06-17,7,1076,47
Argentina,2010-05-26,A,Greece,0,2,World Cup 2010,2,0,2010-06-22,7,1076,13
Argentina,2010-05-26,H,Mexico,3,1,World Cup 2010,3,1,2010-06-27,7,1076,17


In [32]:
test %>%
    mutate(isNeutral = isNeutral_v(country, Opponent, game_name)) %>%
    mutate(Venue = ifelse(isNeutral == 'N', 'N', Venue)) %>%
    select(-c('isNeutral', 'Punkte')) -> final_data_set

In [58]:
final_data_set %>%
    filter(result_left == "") %>%
    filter(game_name == 'World Cup 2022') -> world_cup_2022

final_data_set %>%
    filter(result_left != "") -> data_set_without_wc2022

#data_set_without_wc2022

data_set_without_wc2022 %>%
    mutate(goals_country = as.numeric(goals_country)) %>%
    mutate(Rank = as.numeric(Rang)) %>%
    mutate(goals_country = as.numeric(goals_country)) %>%
    select('country', 'goals_country', 'rank_opponent', 'Rank', 'Venue', 'game_name', 'goals_enemy', 'Opponent', 'date_of_play') -> data_set_without_wc2022_cleaned

In [59]:
data_set_without_wc2022_cleaned

# turn Venue into dummy variables
data_set_without_wc2022_cleaned = dummy_cols(data_set_without_wc2022_cleaned, select_columns = 'Venue')

data_set_without_wc2022_cleaned

data_set_without_wc2022_cleaned %>%
    filter(country == 'Brazil') -> histogramm

country = histogramm$goals_country

mean(country)

var(country)

country,goals_country,rank_opponent,Rank,Venue,game_name,goals_enemy,Opponent,date_of_play
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>
Argentina,3,44,8,H,International Friendlies,2,Costa Rica,2010-01-27
Argentina,2,78,8,H,International Friendlies,1,Jamaica,2010-02-11
Argentina,1,5,9,A,International Friendlies,0,Germany,2010-03-03
Argentina,4,87,7,H,International Friendlies,0,Haiti,2010-05-05
Argentina,5,63,7,H,International Friendlies,0,Canada,2010-05-24
Argentina,1,21,7,N,World Cup 2010,0,Nigeria,2010-06-12
Argentina,4,47,7,N,World Cup 2010,1,South Korea,2010-06-17
Argentina,2,13,7,N,World Cup 2010,0,Greece,2010-06-22
Argentina,3,17,7,N,World Cup 2010,1,Mexico,2010-06-27


country,goals_country,rank_opponent,Rank,Venue,game_name,goals_enemy,Opponent,date_of_play,Venue_A,Venue_H,Venue_N
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<int>,<int>
Argentina,3,44,8,H,International Friendlies,2,Costa Rica,2010-01-27,0,1,0
Argentina,2,78,8,H,International Friendlies,1,Jamaica,2010-02-11,0,1,0
Argentina,1,5,9,A,International Friendlies,0,Germany,2010-03-03,1,0,0
Argentina,4,87,7,H,International Friendlies,0,Haiti,2010-05-05,0,1,0
Argentina,5,63,7,H,International Friendlies,0,Canada,2010-05-24,0,1,0
Argentina,1,21,7,N,World Cup 2010,0,Nigeria,2010-06-12,0,0,1
Argentina,4,47,7,N,World Cup 2010,1,South Korea,2010-06-17,0,0,1
Argentina,2,13,7,N,World Cup 2010,0,Greece,2010-06-22,0,0,1
Argentina,3,17,7,N,World Cup 2010,1,Mexico,2010-06-27,0,0,1


[1] 2.160377

[1] 2.574034

## Split data into test and training data set

In [61]:
#sample = initial_split(data_set_without_wc2022_cleaned, prop=0.7)
#train = training(sample)
#test = testing(sample)

#data_set_without_wc2022_cleaned

data_set_without_wc2022_cleaned %>%
    filter(game_name != 'World Cup 2018') -> train

data_set_without_wc2022_cleaned %>%
    filter(game_name == 'World Cup 2018') -> test

test %>%
    filter(Opponent == 'Uruguay')

country,goals_country,rank_opponent,Rank,Venue,game_name,goals_enemy,Opponent,date_of_play,Venue_A,Venue_H,Venue_N
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<int>,<int>
France,2,14,7,N,World Cup 2018,0,Uruguay,2018-07-06,0,0,1
Portugal,1,14,4,N,World Cup 2018,2,Uruguay,2018-06-30,0,0,1
Saudi Arabia,0,14,67,N,World Cup 2018,1,Uruguay,2018-06-20,0,0,1


In [36]:
test

country,goals_country,rank_opponent,Rank,Venue,game_name,goals_enemy,Opponent,Venue_A,Venue_H,Venue_N
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
Argentina,1,22,5,N,World Cup 2018,1,Iceland,0,0,1
Argentina,0,20,5,N,World Cup 2018,3,Croatia,0,0,1
Argentina,2,48,5,N,World Cup 2018,1,Nigeria,0,0,1
Argentina,3,7,5,N,World Cup 2018,4,France,0,0,1
Australia,1,7,36,N,World Cup 2018,2,France,0,0,1
Australia,1,12,36,N,World Cup 2018,1,Denmark,0,0,1
Australia,0,11,36,N,World Cup 2018,2,Peru,0,0,1
Belgium,3,55,3,N,World Cup 2018,0,Panama,0,0,1
Belgium,5,21,3,N,World Cup 2018,2,Tunisia,0,0,1


In [37]:
train

model = 
    glm(
        goals_country ~ rank_opponent + Venue_H + Venue_N + Rank,
        train,
        family = poisson(link = 'log')
       )
summary(model)

country,goals_country,rank_opponent,Rank,Venue,game_name,goals_enemy,Opponent,Venue_A,Venue_H,Venue_N
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>
Argentina,3,44,8,H,International Friendlies,2,Costa Rica,0,1,0
Argentina,2,78,8,H,International Friendlies,1,Jamaica,0,1,0
Argentina,1,5,9,A,International Friendlies,0,Germany,1,0,0
Argentina,4,87,7,H,International Friendlies,0,Haiti,0,1,0
Argentina,5,63,7,H,International Friendlies,0,Canada,0,1,0
Argentina,1,21,7,N,World Cup 2010,0,Nigeria,0,0,1
Argentina,4,47,7,N,World Cup 2010,1,South Korea,0,0,1
Argentina,2,13,7,N,World Cup 2010,0,Greece,0,0,1
Argentina,3,17,7,N,World Cup 2010,1,Mexico,0,0,1



Call:
glm(formula = goals_country ~ rank_opponent + Venue_H + Venue_N + 
    Rank, family = poisson(link = "log"), data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8204  -1.1880  -0.1984   0.5651   3.7361  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.1922229  0.0401141   4.792 1.65e-06 ***
rank_opponent  0.0070773  0.0004430  15.974  < 2e-16 ***
Venue_H        0.2715692  0.0378653   7.172 7.39e-13 ***
Venue_N        0.1906840  0.0728190   2.619  0.00883 ** 
Rank          -0.0088005  0.0007537 -11.676  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3071.0  on 2123  degrees of freedom
Residual deviance: 2686.8  on 2119  degrees of freedom
  (24 observations deleted due to missingness)
AIC: 6694.8

Number of Fisher Scoring iterations: 5


In [38]:
cum_sum_custom = function(number) {
    return (cumsum(number))
}

cum_sum_custom_v = Vectorize(cum_sum_custom)

In [39]:
getProbabilities = function(model, data, range) {
    dataFrame = data.frame(rank_opponent = data$rank_opponent, Venue_H = data$Venue_H, Venue_N = data$Venue_N, Rank = data$Rank)
    
    lambda = predict(model, newdata = dataFrame, type = "response")
    
    p = Poisson(lambda)
    
    prob = pdf(p, range)
    
    return(prob)
}

In [40]:
getGoals = function(model, data) {
    dataFrame = data.frame(rank_opponent = data$rank_opponent, Venue_H = data$Venue_H, Venue_N = data$Venue_N, Rank = data$Rank)
    
    lambda = predict(model, newdata = dataFrame, type = "response")
    return (rpois(1, lambda))
}

In [51]:
brazil_belgium = data.frame(rank_opponent = 3, Venue_H = 0, Venue_N= 1, Rank = 2)
belgium_brazil = data.frame(rank_opponent = 2, Venue_H = 0, Venue_N= 1, Rank = 3)

france_croatia = data.frame(rank_opponent = 20, Venue_H = 0, Venue_N= 1, Rank = 7)
croatia_france = data.frame(rank_opponent = 7, Venue_H = 0, Venue_N= 1, Rank = 20)

won = 0
lost = 0
draw = 0

for (i in 1:10000) {
    goal1 = getGoals(model, france_croatia)
    goal2 = getGoals(model, croatia_france)

    if (goal1 > goal2) {
        won = won + 1
    } else if (goal1 == goal2){
        draw = draw + 1
    } else {
        lost = lost + 1
    }
}

print(paste0("won: ", won))
print(paste0("lost: ", lost))
print(paste0("draw: ", draw))

[1] "won: 4498"
[1] "lost: 3045"
[1] "draw: 2457"


## Simulation of the World Cup 2018

In [64]:
# Group A (Uruguay, Russia, Saudi Arabia, Egypt)

# Russia and SaudiArabia
russia_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 1, Venue_N= 0, Rank = 70)
saudiArabia_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 67)

# Egypt and Uruguay
egypt_uruguay = data.frame(rank_opponent = 14, Venue_H = 0, Venue_N= 1, Rank = 45)
uruguay_egypt = data.frame(rank_opponent = 45, Venue_H = 0, Venue_N= 1, Rank = 14)

# Russia and Egypt
russia_egypt = data.frame(rank_opponent = 14, Venue_H = 1, Venue_N= 0, Rank = 70)
egypt_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 14)

# Uruguay and Saudi Arabia
uruguay_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 0, Venue_N= 1, Rank = 14)
saudiArabia_uruguay = data.frame(rank_opponent = 14, Venue_H = 0, Venue_N= 1, Rank = 67)

# Uruguay and Russia
russia_uruguay = data.frame(rank_opponent = 14, Venue_H = 1, Venue_N= 0, Rank = 70)
uruguay_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 14)

# Saudi Arabia and Egypt
egypt_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 0, Venue_N= 1, Rank = 45)
saudiArabia_russia = data.frame(rank_opponent = 45, Venue_H = 0, Venue_N= 1, Rank = 67)

# everyone is playing against each other
russia_points = 3
points_against_saudiArabia = getGoals(model, russia_saudiArabia)
print(points_against_saudiArabia)
russia_points = russia_points + points_against_saudiArabia
points_against_egypt = getGoals(model, russia_egypt)
print(points_against_egypt)
russia_points = russia_points + points_against_egypt
points_against_uruguay = getGoals(model, russia_uruguay)
print(points_against_uruguay)
russia_points = russia_points + points_against_uruguay

print(russia_points)


[1] 2
[1] 0
[1] 0
[1] 5


In [47]:
wales_usa = data.frame(rank_opponent = 15, Venue_H = 0, Venue_N= 1, Rank = 18)
usa_wales = data.frame(rank_opponent = 18, Venue_H = 0, Venue_N= 1, Rank = 15)
england_wales = data.frame(rank_opponent = 18, Venue_H = 0, Venue_N = 1, Rank = 5)
wales_england = data.frame(rank_opponent = 5, Venue_H = 0, Venue_N = 1, Rank = 18)

france_croatia = data.frame(rank_opponent = 20, Venue_H = 0, Venue_N= 1, Rank = 7)
croatia_france = data.frame(rank_opponent = 7, Venue_H = 0, Venue_N= 1, Rank = 20)

belgium_england = data.frame(rank_opponent = 13, Venue_H = 0, Venue_N= 1, Rank = 3)
england_belgium = data.frame(rank_opponent = 3, Venue_H = 0, Venue_N= 1, Rank = 13)

russia_saudiArabia = data.frame(rank_opponent = 67, Venue_H = 1, Venue_N= 0, Rank = 70)
saudiArabia_russia = data.frame(rank_opponent = 70, Venue_H = 0, Venue_N= 0, Rank = 67)

brazil_belgium = data.frame(rank_opponent = 3, Venue_H = 0, Venue_N= 1, Rank = 2)
belgium_brazil = data.frame(rank_opponent = 2, Venue_H = 0, Venue_N= 1, Rank = 3)

brazil_mexico = data.frame(rank_opponent = 15, Venue_H = 0, Venue_N= 1, Rank = 2)
mexico_brazil = data.frame(rank_opponent = 2, Venue_H = 0, Venue_N= 1, Rank = 15)



res <- outer(getProbabilities(model, brazil_mexico, 0:6), getProbabilities(model, mexico_brazil, 0:6))
df = data.frame(round(res, digits = 2))

names(df) = c('score_0', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5', 'score_6')

df

df %>%
    mutate(cum_sum = cumsum(score_0) + cumsum(score_1) + cumsum(score_2) + cumsum(score_3) + cumsum(score_4) + cumsum(score_5) + cumsum(score_5))


[1] "Random variable"
[1] 4
[1] "Random variable"
[1] 2


score_0,score_1,score_2,score_3,score_4,score_5,score_6
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.05,0.07,0.05,0.02,0.01,0,0
0.09,0.11,0.07,0.03,0.01,0,0
0.07,0.09,0.06,0.03,0.01,0,0
0.04,0.05,0.03,0.01,0.00,0,0
0.02,0.02,0.01,0.01,0.00,0,0
0.00,0.01,0.00,0.00,0.00,0,0
0.00,0.00,0.00,0.00,0.00,0,0


score_0,score_1,score_2,score_3,score_4,score_5,score_6,cum_sum
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.05,0.07,0.05,0.02,0.01,0,0,0.20
0.09,0.11,0.07,0.03,0.01,0,0,0.51
0.07,0.09,0.06,0.03,0.01,0,0,0.77
0.04,0.05,0.03,0.01,0.00,0,0,0.90
0.02,0.02,0.01,0.01,0.00,0,0,0.96
0.00,0.01,0.00,0.00,0.00,0,0,0.97
0.00,0.00,0.00,0.00,0.00,0,0,0.97


## Write a function that takes the probabilities and calculates the win, loose and draw probabilities

In [182]:
getDraw = function(data) {
    draw = 0
    for (i in 1:length(data[[1]])) {
        draw = draw + data[i, i]
    }
    return (draw)
}

getWin = function(data) {
    win = 0
    for (i in 1:length(data[[1]])) {
        j = 1
        #print("here")
        for (j in (1+i):length(data[[1]])-1) {
            if(j != i) {
                #print(paste0('data[', j, ',', i, ']'))
                #print(data[j, i])
                win = win + data[j, i]
            }
            
        }
    }
    return (win)
}

getLoose = function(data) {
    win = 0
    for (i in 1:length(data[[1]])) {
        j = 1
        for (j in 1:length(data[[1]])) {
            if(j != i & j < i) {
                #print(paste0('data[', j, ',', i, ']'))
                #print(data[j, i])
                win = win + data[j, i]
            }
            
        }
    }
    return (win)
}

In [191]:
df
print("Opponent wins")
getLoose(df)
print("Player wins")
getWin(df)
print("draw")
getDraw(df)

score_0,score_1,score_2,score_3,score_4,score_5,score_6
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.05,0.07,0.05,0.02,0.01,0,0
0.09,0.11,0.07,0.03,0.01,0,0
0.07,0.09,0.06,0.03,0.01,0,0
0.04,0.05,0.03,0.01,0.00,0,0
0.02,0.02,0.01,0.01,0.00,0,0
0.00,0.01,0.00,0.00,0.00,0,0
0.00,0.00,0.00,0.00,0.00,0,0


[1] "Opponent wins"


[1] 0.3

[1] "Player wins"


[1] 0.44

[1] "draw"


[1] 0.23

In [79]:
# Qatar gegen Niederlande

wales_usa = data.frame(rank_opponent = 15, Venue_H = 0, Venue_N= 1, Rank = 18)
usa_wales = data.frame(rank_opponent = 18, Venue_H = 0, Venue_N= 1, Rank = 15)
england_wales = data.frame(rank_opponent = 18, Venue_H = 0, Venue_N = 1, Rank = 5)
wales_england = data.frame(rank_opponent = 5, Venue_H = 0, Venue_N = 1, Rank = 18)

wales_usa_lambda = predict(model, newdata = wales_usa, type = "response")
usa_wales_lambda = predict(model, newdata = usa_wales, type = "response")

wales_england_lambda = predict(model, newdata = wales_england, type = "response")
england_wales_lambda = predict(model, newdata = england_wales, type = "response")

# Poisson Regression anhand des formels f(x) = (lambda^x)*e^(-lambda)/x!
# x = 0 für den USA und x = 0 für Wales
#P_Wales = exp(-tore_wales)
#P_USA = exp(-tore_usa)
#P = P_Wales*P_USA
#P_Wales
#P_USA

options(scipen = 10000, digits = 5)

p_wales_usa = Poisson(wales_usa_lambda)    
prob_a = pdf(p, 0:8)

p_usa_wales = Poisson(usa_wales_lambda)
prob_b = pdf(p, 0:8)

res <- outer(pdf(p_wales_usa, 0:5), pdf(p_usa_wales, 0:5))
df = data.frame(round(res, digits = 5))

names(df) = c('score_0', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5')

df %>%
    mutate(cum_sum = cumsum(score_0) + cumsum(score_1) + cumsum(score_2) + cumsum(score_3) + cumsum(score_4) + cumsum(score_5))

# wales gegen england

p_wales_england = Poisson(wales_england_lambda)    
prob_a = pdf(p, 0:8)

p_england_wales = Poisson(england_wales_lambda)
prob_b = pdf(p, 0:8)

res <- outer(pdf(p_wales_england, 0:5), pdf(p_england_wales, 0:5))
df = data.frame(round(res, digits = 5))

names(df) = c('score_0', 'score_1', 'score_2', 'score_3', 'score_4', 'score_5')

df %>%
    mutate(cum_sum = cumsum(score_0) + cumsum(score_1) + cumsum(score_2) + cumsum(score_3) + cumsum(score_4) + cumsum(score_5))

ERROR: Error in outer(c(pdf(p_wales_usa, 0:5), pdf(p_usa_wales, 0:5))): argument "Y" is missing, with no default
